In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# switch to the project directory
%cd ..
# working directory should be ../pdi

In [ ]:
import sys
import os
module_path = os.path.abspath('src')

if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from pdi.constants import PART_NAME_TO_TARGET_CODE

MODELS = {
    "Attention": "results/attention_hyperparameter_tuning/pion/run_1",
}
target_code = PART_NAME_TO_TARGET_CODE["pion"]

save_dir = "reports"
os.makedirs(save_dir, exist_ok=True)

In [ ]:
import json
from pdi.config import Config
from pdi.engines import build_engine
from pdi.results_and_metrics import TestMetrics
from pdi.data.data_preparation import DataPreparation

last_data_prep: DataPreparation
checksums = set()
test_metrics: dict[str, TestMetrics] = {}
for k, v in MODELS.items():
    with open(f"{v}/config.json", 'r') as f:
        config_data = json.load(f)
    config = Config.from_dict(config_data)
    config.training.device = "cpu"
    engine = build_engine(config, target_code, base_dir=v)
    last_data_prep = engine.get_data_prep()
    checksums.add(last_data_prep._inputs_checksum)
    test_metrics[k] = engine.test(model_dirpath=f"{v}/model_weights").test_metrics

if len(checksums) > 1:
    raise RuntimeError("You shouldn't comapre models trained on different datasets.")

test_metrics["nSigma < 3.0"] = last_data_prep.get_nsigma_test_metrics(target_code)


In [ ]:
import pandas as pd

# Extract metrics from test_results
metrics_data = []
for model_name, test_result in test_metrics.items():
    metrics = test_result.to_dict()  # Assuming `metrics` is a dictionary of test metrics
    metrics['Model'] = model_name
    metrics_data.append(metrics)

# Create a DataFrame for comparison
metrics_df = pd.DataFrame(metrics_data)
print(metrics_df)

# Save as LaTeX table
latex_table = metrics_df.to_latex(index=False)
with open(f"{save_dir}/test_metrics_comparison_{target_code}.tex", "w") as f:
    f.write(latex_table)

# Save as CSV file
metrics_df.to_csv(f"{save_dir}/test_metrics_comparison_{target_code}.csv", index=False)

print("LaTeX table and CSV file saved successfully.")

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve

# Generate precision-recall curves for all models
plt.figure(figsize=(10, 6))
for model_name, test_result in test_metrics.items():
    binary_targets = test_result.targets == test_result.target_code
    precision, recall, thresholds = precision_recall_curve(binary_targets, test_result.predictions)
    plt.plot(recall, precision, label=model_name)

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve for Changing Thresholds")
plt.legend(loc="lower left")
plt.grid()
plt.show()